## 패키지 설치

In [1]:
# 해당 셀을 실행하면 아래의 라이브러리들이 설치됩니다.
# 각 라이브러리가 어떤 용도로 사용되는지 간단히 주석으로 설명합니다.

%pip install reportlab       # PDF 생성 및 조작을 위한 라이브러리
%pip install yfinance        # 야후 파이낸스의 주가 데이터를 손쉽게 불러오기 위한 라이브러리
%pip install matplotlib      # 데이터 시각화를 위한 대표적인 라이브러리
%pip install scrapy          # 웹 크롤링(스크래핑)을 효율적으로 수행하기 위한 프레임워크
%pip install sec_api         # 미국 증권거래위원회(SEC) 관련 데이터 접근을 위한 라이브러리
%pip install langchain       # LLM(대형 언어 모델) 기반 애플리케이션을 빠르게 개발하기 위한 프레임워크
%pip install umap-learn      # 차원 축소, 클러스터링 등에 자주 사용되는 UMAP(Unified Manifold Approximation and Projection) 라이브러리
%pip install scikit-learn    # 머신러닝 기법 구현(회귀, 분류, 클러스터링 등)을 위한 대표적인 라이브러리
%pip install langchain_community  # LangChain 커뮤니티 기반 기능/통합을 제공하는 서드파티 패키지
%pip install tiktoken        # OpenAI GPT 등 토큰화(tokenization) 유틸리티
%pip install langchain-openai  # LangChain과 OpenAI 서비스를 연동하기 위한 확장 패키지
%pip install langchainhub    # 다양한 에이전트, 체인(chain), 프롬프트 템플릿 등을 공유하는 리포지토리
%pip install chromadb        # 벡터 DB를 제공하는 Chroma 라이브러리 (임베딩, 검색, 스토리지 등에 활용)
%pip install langchain-anthropic  # Anthropic(Claude 모델 등)과 LangChain 연동을 위한 라이브러리
%pip install sentence-transformers # 문장의 임베딩을 수행하는 대표적인 패키지
%pip install openbb          # OpenBB Terminal 등 금융 데이터 분석 및 시각화 도구를 제공하는 라이브러리
%pip install googletrans==4.0.0-rc1
%pip install transformers accelerate #라마 설치
%pip install --upgrade httpx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new rele

SyntaxError: unterminated string literal (detected at line 54) (1617074277.py, line 54)

In [6]:
import os
import json
import yfinance as yf
import requests
import csv
from datetime import datetime
from sec_api import ExtractorApi

class ReportAnalysis:
    def __init__(self, ticker_symbol, sec_api_key):
        self.ticker_symbol = ticker_symbol
        self.stock = yf.Ticker(ticker_symbol)
        self.info = self.stock.info
        self.project_dir = f"projects/{ticker_symbol}/"
        self.cache_dir = f"projects/{ticker_symbol}/cache"
        os.makedirs(self.project_dir, exist_ok=True)
        os.makedirs(self.cache_dir, exist_ok=True)
        self.extractor = ExtractorApi(sec_api_key)

    def get_sec_report_addresses_by_date(self, form_type, start_date, end_date):
        endpoint = f"https://api.sec-api.io?token={self.extractor.api_key}"
        query = {
            "query": {
                "query_string": {
                    "query": f"ticker:{self.ticker_symbol} AND formType:\"{form_type}\" AND filedAt:[{start_date} TO {end_date}]"
                }
            },
            "from": "0",
            "size": "100",
            "sort": [{"filedAt": {"order": "desc"}}]
        }
        response = requests.post(endpoint, json=query)
        if response.status_code == 200:
            filings = response.json().get('filings', [])
            return [(f['filedAt'][:10], f['linkToFilingDetails']) for f in filings if 'filedAt' in f and 'linkToFilingDetails' in f]
        else:
            print("Failed to fetch filings.")
            return []

    def save_sections_to_csv(self, form_type, sections, start_date, end_date, csv_filename):
        records = []
        reports = self.get_sec_report_addresses_by_date(form_type, start_date, end_date)
        for filed_at, link in reports:
            for section in sections:
                section_id = str(section).lower()
                if form_type == "10-Q" and not section_id.startswith("part"):
                    section_id = f"part1item{section}" if int(section) <= 4 else f"part2item{int(section) - 4}"
                try:
                    text = self.extractor.get_section(link, section_id, "text")
                    records.append({
                        "Date": filed_at,
                        "Form Type": form_type,
                        "Section": section,
                        "Content": text.replace(" ", " ")  # 내용이 길 경우 앞 3000자만 저장
                    })
                except Exception as e:
                    print(f"Failed to fetch section {section} for {filed_at}: {e}")

        # CSV로 저장
        csv_path = os.path.join(self.project_dir, csv_filename)
        with open(csv_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=["Date", "Form Type", "Section", "Content"])
            writer.writeheader()
            writer.writerows(records)
        print(f"Saved {len(records)} records to {csv_path}")

# 사용 예제
sec_api_key = "62032bfc529611a9a6394f922c7909ab2113e923ce8145b0367f33b5f8701620"  # SEC API 키 입력
ra = ReportAnalysis("TSLA", sec_api_key)

# 2023년 한 해 동안의 10-K와 10-Q 보고서에서 섹션 7과 2를 추출하여 CSV 저장
ra.save_sections_to_csv("10-K", sections=["7"], start_date="2023-01-01", end_date="2023-12-31", csv_filename="10k_sections.csv")
ra.save_sections_to_csv("10-Q", sections=["2"], start_date="2023-01-01", end_date="2023-12-31", csv_filename="10q_sections.csv")


Failed to fetch filings.
Saved 0 records to projects/TSLA/10k_sections.csv
Failed to fetch filings.
Saved 0 records to projects/TSLA/10q_sections.csv
